In [1]:
import re
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import set_seed

set_seed(42) 

/home/gupta.praj/working/aihci/aihci/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA available! Training on GPU.", flush=True)
else:
    device = torch.device('cpu')
    print("CUDA NOT available... Training on CPU.", flush=True)

CUDA available! Training on GPU.


In [3]:
access_token=""

tokenizer = AutoTokenizer.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    token=access_token
)
model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    torch_dtype=torch.bfloat16,
    token=access_token
)

model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  8.32it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [4]:
messages = [
        {"role": "system", "content": "The U.S.-China trade war is an ongoing economic conflict between the world’s two largest economies, characterized by tariffs, trade restrictions, and geopolitical tensions. The U.S. accuses China of unfair trade practices, including intellectual property theft, forced technology transfers, and market manipulation. China, in turn, argues that the U.S. is engaging in protectionism to curb China's economic rise. The dispute has had significant global economic consequences, affecting industries, supply chains, and international trade relationships."},
        {"role": "user", "content": """You are representing the United States in this debate. Your stance is that the U.S. trade policies and tariffs on China are necessary to ensure fair trade and protect American economic interests. Your key arguments include:
 - Intellectual Property Theft and Unfair Trade Practices: China has engaged in widespread intellectual property theft, forced technology transfers, and state-sponsored cyber-espionage, harming U.S. businesses.
 - Trade Imbalance: The U.S. has a massive trade deficit with China due to unfair Chinese policies that limit American exports while promoting Chinese goods in the U.S. market.
 - State Subsidies and Market Manipulation: China heavily subsidizes its industries, creating an uneven playing field that disadvantages U.S. companies.
 - National Security Concerns: Chinese companies, such as Huawei, pose potential security threats due to their ties with the Chinese government, justifying restrictions and sanctions."""},
]

In [5]:
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
# inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_length=2000)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [6]:
def get_prompts_trade_war(country_1, country_2, country_1_citizens, country_2_citizens):
    # Country 1 imposes tarriffs on country 2
    
    system_msg = f"The {country_1}-{country_2} trade war is an ongoing economic conflict between the world’s two largest economies, characterized by tariffs, trade restrictions, and geopolitical tensions. {country_1} accuses {country_2} of unfair trade practices, including intellectual property theft, forced technology transfers, and market manipulation. {country_2}, in turn, argues that {country_1} is engaging in protectionism to curb {country_2}'s economic rise. The dispute has had significant global economic consequences, affecting industries, supply chains, and international trade relationships."

    stance_side_1 = f"""Your stance is that {country_1} trade policies and tariffs on {country_2} are necessary to ensure fair trade and protect {country_1_citizens} economic interests."""

    stance_side_2 = f"""Your stance is that {country_1} is engaging in economic protectionism to suppress {country_2}'s growth and maintain global dominance."""

    side_1_p1 = f"""\n-National Security Concerns: {country_2_citizens} companies pose potential security threats due to their ties with the {country_2_citizens} government, justifying restrictions and sanctions."""

    side_2_p1 = f"""\n-{country_1_citizens} Protectionism and Unilateralism: {country_1} has imposed tariffs and trade restrictions without proper international dialogue, violating principles of free trade."""
    side_2_p2 = f"""\n-International Trade Rules: {country_2} argues that it abides by WTO regulations and that {country_1} should negotiate rather than impose unilateral sanctions."""

    us_pts = f"""\n-Trade Imbalance: The U.S. has a massive trade deficit with China due to unfair chinese policies that limit american exports while promoting chinese goods in the U.S. market.
-State Subsidies and Market Manipulation: China heavily subsidizes its industries, creating an uneven playing field that disadvantages american companies."""

    china_pts = f"""\n-Economic Growth and Competition: China’s rise is based on innovation and development, not unfair practices. The U.S. is trying to block China’s economic progress out of fear of competition.
-Mutual Benefits of Trade: The U.S benefits from cheap chinese goods, and american companies profit from access to China’s massive consumer market."""

    us_instr = "\nYou are the american negotiator shaped by 85 years of RTAA precedent, where tariff reductions must be explicitly reciprocal. Maintain 1934-2025 policy continuity in all proposals."
    china_instr = "\nYou are the chinese negotiator and operate within Deng Xiaoping's 1992 socialist market economy framework, prioritizing technology acquisition through joint ventures over direct transfers."
    
    if country_1 == "U.S.":
        country_1_prompt = """You are representing the U.S. in this debate. """ + stance_side_1 + """ Your key arguments include:""" + us_pts + side_1_p1 + us_instr
    elif country_2 == "U.S.":
        country_2_prompt = "You are representing the U.S. in this debate. " + stance_side_2 + " Your key arguments include:" + us_pts + side_2_p1 + side_2_p2 + us_instr
    if country_1 == "China":
        country_1_prompt = "You are representing China in this debate. " + stance_side_1 + " Your key arguments include:" + china_pts + side_1_p1 + china_instr
    elif country_2 == "China":
        country_2_prompt = "You are representing China in this debate. " + stance_side_2 + " Your key arguments include:" + china_pts + side_2_p1 + side_2_p2 + china_instr
    if country_1 not in ["U.S.", "China"]:
        country_1_prompt = f"""You are representing {country_1} in this debate. Your stance is that {country_1} trade policies and tariffs on {country_2} are necessary to ensure fair trade and protect {country_1_citizens} economic interests. Your key arguments include:
-Trade Imbalance: {country_1} has a massive trade deficit with {country_2} due to unfair {country_2_citizens} policies that limit {country_1_citizens} exports while promoting {country_2_citizens} goods in the {country_1} market.
-State Subsidies and Market Manipulation: {country_2} heavily subsidizes its industries, creating an uneven playing field that disadvantages {country_1_citizens} companies.
-National Security Concerns: {country_2_citizens} companies pose potential security threats due to their ties with the {country_2_citizens} government, justifying restrictions and sanctions."""
    
        country_2_prompt = f"""You are representing {country_2} in this debate. Your stance is that {country_1} is engaging in economic protectionism to suppress {country_2}'s growth and maintain global dominance. Your key arguments include:
-{country_1_citizens} Protectionism and Unilateralism: {country_1} has imposed tariffs and trade restrictions without proper international dialogue, violating principles of free trade.
-Economic Growth and Competition: {country_2}’s rise is based on innovation and development, not unfair practices. {country_1} is trying to block {country_2}’s economic progress out of fear of competition.
-Mutual Benefits of Trade: {country_1} benefits from cheap {country_2_citizens} goods, and {country_1_citizens} companies profit from access to {country_2}’s massive consumer market. A trade war hurts both economies.
-International Trade Rules: {country_2} argues that it abides by WTO regulations and that the {country_1} should negotiate rather than impose unilateral sanctions."""

    return system_msg, country_1_prompt, country_2_prompt

In [8]:
country_1 = "U.S."
country_2 = "China"
country_1_citizens = "american"
country_2_citizens = "chinese"

In [9]:
system_msg, country_1_prompt, country_2_prompt = get_prompts_trade_war(country_1, country_2, country_1_citizens, country_2_citizens)

In [10]:
system_msg 

"The U.S.-China trade war is an ongoing economic conflict between the world’s two largest economies, characterized by tariffs, trade restrictions, and geopolitical tensions. U.S. accuses China of unfair trade practices, including intellectual property theft, forced technology transfers, and market manipulation. China, in turn, argues that U.S. is engaging in protectionism to curb China's economic rise. The dispute has had significant global economic consequences, affecting industries, supply chains, and international trade relationships."

In [11]:
country_1_prompt

'You are representing the U.S. in this debate. Your stance is that U.S. trade policies and tariffs on China are necessary to ensure fair trade and protect american economic interests. Your key arguments include:\n-Trade Imbalance: The U.S. has a massive trade deficit with China due to unfair chinese policies that limit american exports while promoting chinese goods in the U.S. market.\n-State Subsidies and Market Manipulation: China heavily subsidizes its industries, creating an uneven playing field that disadvantages american companies.\n-National Security Concerns: chinese companies pose potential security threats due to their ties with the chinese government, justifying restrictions and sanctions.\nYou are the american negotiator shaped by 85 years of RTAA precedent, where tariff reductions must be explicitly reciprocal. Maintain 1934-2025 policy continuity in all proposals.'

In [12]:
country_2_prompt 

"You are representing China in this debate. Your stance is that U.S. is engaging in economic protectionism to suppress China's growth and maintain global dominance. Your key arguments include:\n-Economic Growth and Competition: China’s rise is based on innovation and development, not unfair practices. The U.S. is trying to block China’s economic progress out of fear of competition.\n-Mutual Benefits of Trade: The U.S benefits from cheap chinese goods, and american companies profit from access to China’s massive consumer market.\n-american Protectionism and Unilateralism: U.S. has imposed tariffs and trade restrictions without proper international dialogue, violating principles of free trade.\n-International Trade Rules: China argues that it abides by WTO regulations and that U.S. should negotiate rather than impose unilateral sanctions.\nYou are the chinese negotiator and operate within Deng Xiaoping's 1992 socialist market economy framework, prioritizing technology acquisition through

In [13]:
def get_switched_outputs_trade_war(country_1, country_2, country_1_citizens, country_2_citizens):
    # country 1 imposed tarriffs
    system_msg, country_1_prompt_side1, country_2_prompt_side2 = get_prompts_trade_war(country_1, country_2, country_1_citizens, country_2_citizens) 
    print(system_msg)
    print('\n\n')
    print(country_1_prompt_side1)
    print('\n\n')
    print(country_2_prompt_side2)
    messages_c1 = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": country_1_prompt_side1},
    ]
    inputs = tokenizer.apply_chat_template(messages_c1, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=2000)
    c1_side1 = tokenizer.decode(outputs[0])

    messages_c2 = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": country_2_prompt_side2},
    ]
    inputs = tokenizer.apply_chat_template(messages_c2, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=2000)
    c2_side2 = tokenizer.decode(outputs[0])

    print('\n\n')
    print('\n\n')
    print(c1_side1)
    print('\n\n')
    print(c2_side2)
    print('\n\n')
    print('\n\n')
    print('\n\n')
    print('\n\n')
    
    # country 2 imposed tarriffs
    system_msg_sw, country_2_prompt_side1, country_1_prompt_side2 = get_prompts_trade_war(country_2, country_1, country_2_citizens, country_1_citizens) 
    print(system_msg_sw)
    print('\n\n')
    print(country_1_prompt_side2)
    print('\n\n')
    print(country_2_prompt_side1)
    
    messages_c1 = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": country_1_prompt_side2},
    ]
    inputs = tokenizer.apply_chat_template(messages_c1, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=5000)
    c1_side2 = tokenizer.decode(outputs[0])

    messages_c2 = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": country_2_prompt_side1},
    ]
    inputs = tokenizer.apply_chat_template(messages_c2, add_generation_prompt=True, tokenize=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_length=5000)
    c2_side1 = tokenizer.decode(outputs[0])

    print('\n\n')
    print('\n\n')
    print(c1_side2)
    print('\n\n')
    print(c2_side1)
    print('\n\n')
    print('\n\n')
    print('\n\n')
    print('\n\n')

In [14]:
get_switched_outputs_trade_war(country_1, country_2, country_1_citizens, country_2_citizens)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The U.S.-China trade war is an ongoing economic conflict between the world’s two largest economies, characterized by tariffs, trade restrictions, and geopolitical tensions. U.S. accuses China of unfair trade practices, including intellectual property theft, forced technology transfers, and market manipulation. China, in turn, argues that U.S. is engaging in protectionism to curb China's economic rise. The dispute has had significant global economic consequences, affecting industries, supply chains, and international trade relationships.



You are representing the U.S. in this debate. Your stance is that U.S. trade policies and tariffs on China are necessary to ensure fair trade and protect american economic interests. Your key arguments include:
-Trade Imbalance: The U.S. has a massive trade deficit with China due to unfair chinese policies that limit american exports while promoting chinese goods in the U.S. market.
-State Subsidies and Market Manipulation: China heavily subsidizes i

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.








<｜begin▁of▁sentence｜>The U.S.-China trade war is an ongoing economic conflict between the world’s two largest economies, characterized by tariffs, trade restrictions, and geopolitical tensions. U.S. accuses China of unfair trade practices, including intellectual property theft, forced technology transfers, and market manipulation. China, in turn, argues that U.S. is engaging in protectionism to curb China's economic rise. The dispute has had significant global economic consequences, affecting industries, supply chains, and international trade relationships.<｜User｜>You are representing the U.S. in this debate. Your stance is that U.S. trade policies and tariffs on China are necessary to ensure fair trade and protect american economic interests. Your key arguments include:
-Trade Imbalance: The U.S. has a massive trade deficit with China due to unfair chinese policies that limit american exports while promoting chinese goods in the U.S. market.
-State Subsidies and Market Manipulat

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.








<｜begin▁of▁sentence｜>The U.S.-China trade war is an ongoing economic conflict between the world’s two largest economies, characterized by tariffs, trade restrictions, and geopolitical tensions. U.S. accuses China of unfair trade practices, including intellectual property theft, forced technology transfers, and market manipulation. China, in turn, argues that U.S. is engaging in protectionism to curb China's economic rise. The dispute has had significant global economic consequences, affecting industries, supply chains, and international trade relationships.<｜User｜>You are representing the U.S. in this debate. Your stance is that China is engaging in economic protectionism to suppress U.S.'s growth and maintain global dominance. Your key arguments include:
-Trade Imbalance: The U.S. has a massive trade deficit with China due to unfair chinese policies that limit american exports while promoting chinese goods in the U.S. market.
-State Subsidies and Market Manipulation: China heavi